<h1><b>Title : </b></h1>

<h1><b>Introduction:</b></h1>

<h3> Imports:</h3>

In [27]:
import pickle
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as ss
from pandas.plotting import scatter_matrix
import seaborn as sns
import pandas.testing as pdt
import scipy.stats as ss
import numpy.testing as npt
import pandas.testing as pdt

sns.set_style("darkgrid")

<h3>Reading in data:</h3>

In [28]:
data = pd.read_excel('valuation-2010.xls')

<h1><b>Data Description:</b></h1>

In [29]:
data.describe()['Main Roll No']

count    57246.0
mean         2.0
std          0.0
min          2.0
25%          2.0
50%          2.0
75%          2.0
max          2.0
Name: Main Roll No, dtype: float64

In [30]:
data.head()

,Township,Stand No,Sect Title No,StandDescrip,Owner Name,Account No,Prop Category,TariffCode,Physical Addr,Street No,...,BC Create Date,BC Orig Date,BC Altern Date,BC Value,BC Tariff,Deed Date,Deed No,Register Date,Register No,Capture Date
0,NELSPRUIT TOWN,000000000000010000100000000,NaN,GED 1 ERF 1,NELSPRUIT HARDWARE,2218141.0,BUS,VABUS,PAUL KRUGER STREET,1,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,19810925.0,T46343/81,99999999.0
1,NELSPRUIT TOWN,00000000000001000RE00000000,NaN,RESTANT VAN ERF 1,MINTY IE & SONS PTY LTD,2218119.0,BUS,VABUS,ANDERSON STREET,36,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,19810925.0,T46344/81,99999999.0
2,NELSPRUIT TOWN,000000000000020000000000000,NaN,ERF 2,MINTYS UNIVERSAL STORES LTD,2218130.0,BUS,VABUS,ANDREW STREET,33,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,T96299/200,0.0
3,NELSPRUIT TOWN,000000000000030000000000000,NaN,ERF 3,LOWMIN PROPERTIES (PTY) LTD,2217568.0,BUS,VABUS,ANDERSON STREET,32,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,T96299/200,0.0
4,NELSPRUIT TOWN,000000000000040000000000000,NaN,ERF 4,NELMIN ERF 4 PROPERTIES (PTY) LTD,2040436.0,BUS,VABUS,ANDERSON STREET,30,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,20020814.0,T96299/200,20021029.0


<h1><b>Data processing:</b></h1>

In [5]:
#Columns to be removed
#Stand No, Sect Tittle, Account Number, TariffCode, Street No, Up-Level Size,
#Exemp Size, Main Roll No, Owner Type, Valuation Date, Up-levelInd, Usage Description
#Zone Description, SG Number, Status, Postal Address(1,2, 3, 4, 5), All columns with 'BC', Postal Code,
#Deed No, Register No, Valuation Date, Capture Date 
# Stand No, Sect Tittle, Account Number, TarrifCode(Because it is dependent ), Street No,Up-Level Size,  Exemp Size,Exemp Value,  
#Main Roll No, Owner Type(dependent on), Valuation date, Up-LevelInd, usage description, Zone Description, SGNumber, 
# Status, (Postal Adrress1, 2, 3, 4, 5), BC Date, Postal Code, Deed No,(All BC Values), Register No.
#Valuation Date(date time type), Capture Date
# Usage Code: Discard later if needed
# What does Empty Stand Ind - indicate?(remove if possible)
# Investigate the meaning of Valuation Type Values
#More info on: Meter Book
# Verify if Postal Code matches with the township
# 
data_new = data.drop(columns = ['Stand No', 'Sect Title No', 'Account No', 'TariffCode',
       'Street No', 'Up-Level Size','Exemp Size',
       'Exemp Value', 'Main Roll No', 'Owner Type',
       'Valuation Date', 'Up-LevelInd','BC Create Date',
       'BC Orig Date', 'BC Altern Date', 'BC Value','BC Tariff',
       'Zone Descrip', 'UsageDescrip','SGNumber','Status',
       'Postal Addr1', 'Postal Addr2', 'Postal Addr3',
       'Postal Addr4', 'Postal Addr5', 'Postal Code',
       'Deed Date', 'Deed No', 'Register Date', 'Register No'])
data_new = data_new.drop(columns = [data_new.columns[-1]])
#Capute date is not properly named, so we must remove it using the indexs
data_new.head()

,Township,StandDescrip,Owner Name,Prop Category,Physical Addr,Size,MarketValue,Supp RollNo,Effect Date,Zone Code,Usage Code,Empty Stand Ind,Valuation Type,Ward,Cycle,Meter Book,Val Cnt
0,NELSPRUIT TOWN,GED 1 ERF 1,NELSPRUIT HARDWARE,BUS,PAUL KRUGER STREET,862.0,2900000.0,4.0,20090805.0,BUS2,BUS2,N,VI,14.0,3.0,MB0201,1.0
1,NELSPRUIT TOWN,RESTANT VAN ERF 1,MINTY IE & SONS PTY LTD,BUS,ANDERSON STREET,625.0,1700000.0,1.0,20090701.0,BUS2,BUS2,NaN,VG,14.0,3.0,MB0005,1.0
2,NELSPRUIT TOWN,ERF 2,MINTYS UNIVERSAL STORES LTD,BUS,ANDREW STREET,1487.0,6200000.0,4.0,20090805.0,BUS2,BUS2,NaN,VI,14.0,3.0,MB0201,1.0
3,NELSPRUIT TOWN,ERF 3,LOWMIN PROPERTIES (PTY) LTD,BUS,ANDERSON STREET,1487.0,7300000.0,4.0,20090805.0,BUS2,BUS2,NaN,VI,14.0,3.0,MB0005,1.0
4,NELSPRUIT TOWN,ERF 4,NELMIN ERF 4 PROPERTIES (PTY) LTD,BUS,ANDERSON STREET,1487.0,4500000.0,6.0,20100125.0,BUS2,BUS2,N,VI,14.0,3.0,MB0005,1.0


In [6]:
my_data_set = data_new.iloc[:,7:14]
my_data_set

,Supp RollNo,Effect Date,Zone Code,Usage Code,Empty Stand Ind,Valuation Type,Ward
0,4.0,20090805.0,BUS2,BUS2,N,VI,14.0
1,1.0,20090701.0,BUS2,BUS2,NaN,VG,14.0
2,4.0,20090805.0,BUS2,BUS2,NaN,VI,14.0
3,4.0,20090805.0,BUS2,BUS2,NaN,VI,14.0
4,6.0,20100125.0,BUS2,BUS2,N,VI,14.0
...,...,...,...,...,...,...,...
57247,1.0,20090701.0,RES2,RES2,N,VG,30.0
57248,1.0,20090701.0,RES2,RES2,N,VG,30.0
57249,1.0,20090701.0,RES2,RES2,N,VG,30.0
57250,2.0,20090702.0,RES1,RES1,N,VI,30.0


In [7]:
#There is only N and null values in 'Empty Stand Ind'. The question is 'is null equal to yes?', this assumption is far fetched thus might be incorrect, based on that we take out this column
my_data_set = my_data_set.drop(columns = 'Empty Stand Ind')
my_data_set

,Supp RollNo,Effect Date,Zone Code,Usage Code,Valuation Type,Ward
0,4.0,20090805.0,BUS2,BUS2,VI,14.0
1,1.0,20090701.0,BUS2,BUS2,VG,14.0
2,4.0,20090805.0,BUS2,BUS2,VI,14.0
3,4.0,20090805.0,BUS2,BUS2,VI,14.0
4,6.0,20100125.0,BUS2,BUS2,VI,14.0
...,...,...,...,...,...,...
57247,1.0,20090701.0,RES2,RES2,VG,30.0
57248,1.0,20090701.0,RES2,RES2,VG,30.0
57249,1.0,20090701.0,RES2,RES2,VG,30.0
57250,2.0,20090702.0,RES1,RES1,VI,30.0


In [8]:
#Checking columns with null values
ncols_missing = my_data_set.isnull().any(axis=0).sum()
print(ncols_missing)
#Checking rows with null values
nrows_missing = my_data_set.isnull().any(axis=1).sum()
print(nrows_missing)

6
6


In [9]:
#All 6 columns have missing values
#6 rows have missing values

#For seing which rows index are null in respect to each column.
print(np.where(my_data_set['Supp RollNo'].isnull() == True)[0])
print("rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Supp RollNo")
print(np.where(my_data_set['Effect Date'].isnull() == True)[0])
print("rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Effect Date")
print(np.where(my_data_set['Zone Code'].isnull() == True)[0])
print("rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Zone Code")
print(np.where(my_data_set['Usage Code'].isnull() == True)[0])
print("rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Usage Code")
print(np.where(my_data_set['Valuation Type'].isnull() == True)[0])
print("rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Valuation Type")
print(np.where(my_data_set['Ward'].isnull() == True)[0])
print("rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Ward")

[30400 40754 40767 40800 40815 40834]
rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Supp RollNo
[30400 40754 40767 40800 40815 40834]
rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Effect Date
[30400 40754 40767 40800 40815 40834]
rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Zone Code
[30400 40754 40767 40800 40815 40834]
rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Usage Code
[30400 40754 40767 40800 40815 40834]
rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Valuation Type
[30400 40754 40767 40800 40815 40834]
rows 30400, 40754, 40767, 40800, 40815 and 40834 are null for Ward


In [10]:
data_new.loc[[30400, 40754, 40767, 40800, 40815, 40834]]
#All the rows with missing values are removed
data_new = data_new.drop([30400, 40754, 40767, 40800, 40815, 40834]);
data_new = data_new.drop(columns = 'Empty Stand Ind')

In [11]:
my_new_data_set = data_new.iloc[:,7:14]
#Checking columns with null values
ncols_missing = my_new_data_set.isnull().any(axis=0).sum()
print(ncols_missing)
#Checking rows with null values
nrows_missing = my_new_data_set.isnull().any(axis=1).sum()
print(nrows_missing)

0
0


In [12]:
#Making my dictionaries
dic = pd.Series(my_new_data_set['Zone Code'].unique())
dic.index = dic.index + 1
dic = pd.Series(dic.index.values, index = dic)

dic2 = pd.Series(my_new_data_set['Usage Code'].unique())
dic2.index = dic2.index + 1
dic2 = pd.Series(dic2.index.values, index = dic2)

dic3 = pd.Series(my_new_data_set['Valuation Type'].unique())
dic3.index = dic3.index + 1
dic3 = pd.Series(dic3.index.values, index = dic3)
#dic3

In [13]:
#Encoding based on the dictionaries
lookup_table = dict(dic)

def lookup_customer(val):
    return lookup_table.get(val, np.nan)


my_new_data_set['Zone Code'] = my_new_data_set['Zone Code'].apply(lambda x: lookup_customer(x));

lookup_table = dict(dic2)
my_new_data_set['Usage Code'] = my_new_data_set['Usage Code'].apply(lambda x: lookup_customer(x));

lookup_table = dict(dic3)
my_new_data_set['Valuation Type'] = my_new_data_set['Valuation Type'].apply(lambda x: lookup_customer(x));

my_new_data_set
#df_new.head(20)

,Supp RollNo,Effect Date,Zone Code,Usage Code,Valuation Type,Ward,Cycle
0,4.0,20090805.0,1,1,1,14.0,3.0
1,1.0,20090701.0,1,1,2,14.0,3.0
2,4.0,20090805.0,1,1,1,14.0,3.0
3,4.0,20090805.0,1,1,1,14.0,3.0
4,6.0,20100125.0,1,1,1,14.0,3.0
...,...,...,...,...,...,...,...
57247,1.0,20090701.0,14,15,2,30.0,40.0
57248,1.0,20090701.0,14,15,2,30.0,40.0
57249,1.0,20090701.0,14,15,2,30.0,40.0
57250,2.0,20090702.0,9,9,1,30.0,40.0


In [14]:
###Supp RollNo is Supplimentary roll number
##Every property in every municipality should (hypothetically) be on a roll, but because properties are continuously coming into existence and ceasing to exist, new rolls are created (these are referred to as supplementary rolls) to include any properties that have not been previously recorded on another, prior, general roll.
my_new_data_set.info()
my_new_data_set['Supp RollNo'].unique()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57246 entries, 0 to 57251
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Supp RollNo     57246 non-null  float64
 1   Effect Date     57246 non-null  float64
 2   Zone Code       57246 non-null  int64  
 3   Usage Code      57246 non-null  int64  
 4   Valuation Type  57246 non-null  int64  
 5   Ward            57246 non-null  float64
 6   Cycle           57246 non-null  float64
dtypes: float64(4), int64(3)
memory usage: 3.5 MB


array([4., 1., 6., 5., 3., 2.])

In [15]:
#convert column Effect date to date time
my_new_data_set['Effect Date'] = pd.to_datetime(my_new_data_set['Effect Date'], format='%Y%m%d') 
my_new_data_set

,Supp RollNo,Effect Date,Zone Code,Usage Code,Valuation Type,Ward,Cycle
0,4.0,2009-08-05,1,1,1,14.0,3.0
1,1.0,2009-07-01,1,1,2,14.0,3.0
2,4.0,2009-08-05,1,1,1,14.0,3.0
3,4.0,2009-08-05,1,1,1,14.0,3.0
4,6.0,2010-01-25,1,1,1,14.0,3.0
...,...,...,...,...,...,...,...
57247,1.0,2009-07-01,14,15,2,30.0,40.0
57248,1.0,2009-07-01,14,15,2,30.0,40.0
57249,1.0,2009-07-01,14,15,2,30.0,40.0
57250,2.0,2009-07-02,9,9,1,30.0,40.0


In [16]:
my_new_data_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57246 entries, 0 to 57251
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Supp RollNo     57246 non-null  float64       
 1   Effect Date     57246 non-null  datetime64[ns]
 2   Zone Code       57246 non-null  int64         
 3   Usage Code      57246 non-null  int64         
 4   Valuation Type  57246 non-null  int64         
 5   Ward            57246 non-null  float64       
 6   Cycle           57246 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(3)
memory usage: 3.5 MB


In [17]:
#Note that ERF is afrikaans and it means inherit
#Restant mean remaining when translated from afrikaans to english
#Restant van Erf therefore means remaing of land

In [18]:
df = pd.DataFrame()
df = data_new.copy()
df

,Township,StandDescrip,Owner Name,Prop Category,Physical Addr,Size,MarketValue,Supp RollNo,Effect Date,Zone Code,Usage Code,Valuation Type,Ward,Cycle,Meter Book,Val Cnt
0,NELSPRUIT TOWN,GED 1 ERF 1,NELSPRUIT HARDWARE,BUS,PAUL KRUGER STREET,862.0,2900000.0,4.0,20090805.0,BUS2,BUS2,VI,14.0,3.0,MB0201,1.0
1,NELSPRUIT TOWN,RESTANT VAN ERF 1,MINTY IE & SONS PTY LTD,BUS,ANDERSON STREET,625.0,1700000.0,1.0,20090701.0,BUS2,BUS2,VG,14.0,3.0,MB0005,1.0
2,NELSPRUIT TOWN,ERF 2,MINTYS UNIVERSAL STORES LTD,BUS,ANDREW STREET,1487.0,6200000.0,4.0,20090805.0,BUS2,BUS2,VI,14.0,3.0,MB0201,1.0
3,NELSPRUIT TOWN,ERF 3,LOWMIN PROPERTIES (PTY) LTD,BUS,ANDERSON STREET,1487.0,7300000.0,4.0,20090805.0,BUS2,BUS2,VI,14.0,3.0,MB0005,1.0
4,NELSPRUIT TOWN,ERF 4,NELMIN ERF 4 PROPERTIES (PTY) LTD,BUS,ANDERSON STREET,1487.0,4500000.0,6.0,20100125.0,BUS2,BUS2,VI,14.0,3.0,MB0005,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57247,WHITERIVER ACRES,ERF 00000056,INGWE TRUST,RES,PETER GRAHAM,91.0,475000.0,1.0,20090701.0,RES2,RES2,VG,30.0,40.0,MB4207,1.0
57248,WHITERIVER ACRES,ERF 00000056,S EIGENHUIS,RES,PETER GRAHAM,91.0,475000.0,1.0,20090701.0,RES2,RES2,VG,30.0,40.0,MB4207,1.0
57249,WHITERIVER ACRES,ERF 00000056,JS STANDER,RES,PETER GRAHAM,91.0,475000.0,1.0,20090701.0,RES2,RES2,VG,30.0,40.0,MB4207,1.0
57250,WILLIAM LYNNSTRAAT 51,PORTION 00005 OF ERF 00000332,JS STANDER,RES,WILLIAM LYNN STREET,105.0,459000.0,2.0,20090702.0,RES1,RES1,VI,30.0,40.0,MB4221,1.0


In [19]:
print(df['Township'].isnull().sum())               #printing null values of Township
print(df['StandDescrip'].isnull().sum())           #printing null values of StandDescrip
print(df['Owner Name'].isnull().sum())             #printing null values of Owner Name
print(df['Prop Category'].isnull().sum())          #printing null values of Prop Category
print(df['Physical Addr'].isnull().sum())          #printing null values of Physical Addr
print(df['Size'].isnull().sum())                   #printing null values of Size
print(df['MarketValue'].isnull().sum())            #printing null values of MarketValue

0
28609
13
0
279
0
0


In [20]:
#encode prop category
#1 BUS and #2 RES #3 GOV #4 MUN #5 POW #6 OTH #7 PSI #8 AG #9 RUR
df['Prop Category'] = np.where(df['Prop Category'] == 'BUS', 1 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'RES', 2 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'GOV', 3 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'MUN', 4 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'POW', 5 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'OTH', 6 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'PSI', 7 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'AG',  8 ,df['Prop Category'])
df['Prop Category'] = np.where(df['Prop Category'] == 'RUR', 9 ,df['Prop Category'])

In [21]:
#encode missing owners name with 'other'
df['Owner Name'] = df['Owner Name'].fillna('other')
df['Owner Name'].isnull().sum()

0

In [22]:
#only managed to recover 1059 StandDescrip values
def lookup_stand_desc(stand_no):
    return dictionary.get(stand_no)

df1 = data.copy()
stand_desc = df['StandDescrip'].tolist() + ['nan', 'nan', 'nan', 'nan', 'nan', 'nan']
series = pd.Series(stand_desc, index =df1['Stand No']).dropna()
dictionary = series.to_dict()

df['StandDescrip'] = df1['Stand No'].apply(lookup_stand_desc)

In [23]:
df['StandDescrip'].isnull().sum()

27550

In [24]:
#encode missing values for physical address
df['Physical Addr'] = df['Physical Addr'].fillna('Some Street')
df['Physical Addr'].isnull().sum()

0

In [25]:
physical_addr = df['Physical Addr'].tolist()
streets  = []
street_num  = 1
for i, street in enumerate(physical_addr):
    if street not in streets:
        streets.append(street)
        physical_addr[i] = streets.index(street) + 1
    else:
        physical_addr[i] = streets.index(street) + 1
df['Physical Addr'] = physical_addr